In [2]:
# Import dependencies

import pandas as pd
import requests
import gmaps
import numpy as np
from config import g_key



In [3]:
gmaps.configure(api_key=g_key)

In [4]:
weather_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
weather_df.head()

,City ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,288.64,71,56,3.71,broken clouds
1,1,Castro,BR,-24.7911,-50.0119,287.77,95,18,2.68,few clouds
2,2,Busselton,AU,-33.6500,115.3333,286.94,86,0,3.56,clear sky
3,3,Zhanaozen,KZ,43.3000,52.8000,284.01,52,70,0.61,broken clouds
4,4,Mataura,NZ,-46.1927,168.8643,286.19,74,5,1.01,clear sky


In [5]:
weather_F_conversion = [(temp-273.15) * (9/5) + 32 for temp in weather_df['Max Temp']]


In [6]:
weather_df.loc[:, 'Max Temp'] = weather_F_conversion

weather_df.head()

,City ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.882,71,56,3.71,broken clouds
1,1,Castro,BR,-24.7911,-50.0119,58.316,95,18,2.68,few clouds
2,2,Busselton,AU,-33.6500,115.3333,56.822,86,0,3.56,clear sky
3,3,Zhanaozen,KZ,43.3000,52.8000,51.548,52,70,0.61,broken clouds
4,4,Mataura,NZ,-46.1927,168.8643,55.472,74,5,1.01,clear sky


In [27]:
min_temp = float(input('What is the minimum preferable temperature for your trip?'))
max_temp = float(input('What is the maximum preferable temperature for your trip?'))

In [46]:
weather_pref_df = weather_df.loc[(weather_df['Max Temp'] >= min_temp) & \
                                 (weather_df['Max Temp'] <= max_temp)]



In [47]:
weather_pref_df.count()

City ID                205
City                   205
Country                205
Lat                    205
Long                   205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [48]:
weather_pref_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Long                   0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [92]:
hotel_df = weather_pref_df[['City', 'Country', 'Max Temp','Current Description', 'Lat', 'Long']].copy()

hotel_df['Hotel Name'] = ''


In [93]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
6,Cayenne,GF,82.094,broken clouds,4.9333,-52.3333,
13,Vaini,TO,84.362,scattered clouds,-21.2000,-175.2000,
17,Manaure,CO,87.566,few clouds,11.7751,-72.4445,
21,Ibra,OM,77.558,clear sky,22.6906,58.5334,
22,Beira,MZ,78.890,broken clouds,-19.8436,34.8389,


In [94]:
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [95]:
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lon = row['Long']

    params['location'] = f'{lat},{lon}'

    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    hotels = requests.get(url, params=params).json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']

    except (IndexError):
        print('Hotel not found...skipping.')


Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [96]:
hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
6,Cayenne,GF,82.094,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
13,Vaini,TO,84.362,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
17,Manaure,CO,87.566,few clouds,11.7751,-72.4445,Hotel Palaaima
21,Ibra,OM,77.558,clear sky,22.6906,58.5334,مزرعة راحة النفوس المنزفة
22,Beira,MZ,78.890,broken clouds,-19.8436,34.8389,Beira Terrace Hotel
23,Rikitea,PF,77.828,light rain,-23.1203,-134.9692,People ThankYou
24,Grand Gaube,MU,78.386,few clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
31,Boa Vista,BR,89.582,broken clouds,2.8197,-60.6733,Aipana Plaza Hotel
33,Kapaa,US,82.382,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
37,Soyo,AO,78.476,overcast clouds,-6.1349,12.3689,Hotel Mpampa


In [99]:
hotel_df.loc[hotel_df['Hotel Name'] == ''] = np.nan

hotel_df.dropna(inplace=True)

In [8]:
hotel_df.head(30)

NameError: name 'hotel_df' is not defined

In [101]:
hotel_df.to_csv('WeatherPy_vacation.csv')

In [11]:
hotel_df = pd.read_csv('WeatherPy_vacation.csv')

hotel_df.head(10)

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
0,6,Cayenne,GF,82.094,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
1,13,Vaini,TO,84.362,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
2,17,Manaure,CO,87.566,few clouds,11.7751,-72.4445,Hotel Palaaima
3,21,Ibra,OM,77.558,clear sky,22.6906,58.5334,مزرعة راحة النفوس المنزفة
4,22,Beira,MZ,78.890,broken clouds,-19.8436,34.8389,Beira Terrace Hotel
5,23,Rikitea,PF,77.828,light rain,-23.1203,-134.9692,People ThankYou
6,24,Grand Gaube,MU,78.386,few clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
7,31,Boa Vista,BR,89.582,broken clouds,2.8197,-60.6733,Aipana Plaza Hotel
8,33,Kapaa,US,82.382,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,37,Soyo,AO,78.476,overcast clouds,-6.1349,12.3689,Hotel Mpampa


In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

In [18]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
locations = hotel_df[['Lat', 'Long']]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))